In [59]:
import pandas as pd
import numpy as np
import pymysql
import re

In [12]:
conn = pymysql.connect(host = '58.123.162.40', user = 'woorisool', password = 'Dnfltnf12!@', db = 'woorisool', 
                       charset = 'utf8', 
                       autocommit=True, 
                       cursorclass=pymysql.cursors.DictCursor)

In [13]:
cur = conn.cursor()

In [47]:
sql_join = "SELECT \
            prdt.PRODUCT_ID, PRODUCT_NAME, prdt.TYPE_ID, TYPE_NAME, PRODUCT_PROOF, INFO_KEYWORD \
            FROM tb_product AS prdt \
            JOIN tb_info AS info ON prdt.PRODUCT_ID = info.PRODUCT_ID \
            JOIN tb_brewery AS brew ON prdt.BREWERY_ID = brew.BREWERY_ID \
            JOIN tb_type AS tp ON prdt.TYPE_ID = tp.TYPE_ID"

cur.execute(sql_join)

rows = cur.fetchall()
print(rows)

tb_join = pd.DataFrame(rows)

[{'PRODUCT_ID': '10001', 'PRODUCT_NAME': '1000억 유산균 막걸리', 'TYPE_ID': '10000', 'TYPE_NAME': '탁주', 'PRODUCT_PROOF': 5.0, 'INFO_KEYWORD': '산미 개운하고 식물성 유산균 청량한 국순당 강화'}, {'PRODUCT_ID': '10002', 'PRODUCT_NAME': '1000억 프리바이오 막걸리', 'TYPE_ID': '10000', 'TYPE_NAME': '탁주', 'PRODUCT_PROOF': 5.0, 'INFO_KEYWORD': '활발한 작용 대한 억제 성분 유익 프리바이오틱스 유산균 장내 유해 활동 관심 청량감'}, {'PRODUCT_ID': '10003', 'PRODUCT_NAME': '109(백구) 막걸리', 'TYPE_ID': '10000', 'TYPE_NAME': '탁주', 'PRODUCT_PROOF': 8.0, 'INFO_KEYWORD': '백구 강아지 달콤하면서 색다른 고소한 유기견 크라운 시나몬 심볼 후원'}, {'PRODUCT_ID': '10004', 'PRODUCT_NAME': '가야 프리미엄 막걸리', 'TYPE_ID': '10000', 'TYPE_NAME': '탁주', 'PRODUCT_PROOF': 6.0, 'INFO_KEYWORD': '김해 전혀 가야 달콤한 조량'}, {'PRODUCT_ID': '10005', 'PRODUCT_NAME': '가와지탁주 7.5 ', 'TYPE_ID': '10000', 'TYPE_NAME': '탁주', 'PRODUCT_PROOF': 7.5, 'INFO_KEYWORD': '산뜻한 탄산 대화 아름다운 물씬 미세한 텍스쳐 산미 사과 자부심 형상 지라 바나나 개운한 바닐라 과일 특화 대한 적절한 가벼운 멜론 껍질 쌀알 은은히 포도 깔끔하고 고양시 생산자 애정 묵직한'}, {'PRODUCT_ID': '10006', 'PRODUCT_NAME': '가와지탁주 9.5', 'TYPE_ID': '10000', 'TYPE

In [48]:
tb_join

,PRODUCT_ID,PRODUCT_NAME,TYPE_ID,TYPE_NAME,PRODUCT_PROOF,INFO_KEYWORD
0,10001,1000억 유산균 막걸리,10000,탁주,5.0,산미 개운하고 식물성 유산균 청량한 국순당 강화
1,10002,1000억 프리바이오 막걸리,10000,탁주,5.0,활발한 작용 대한 억제 성분 유익 프리바이오틱스 유산균 장내 유해 활동 관심 청량감
2,10003,109(백구) 막걸리,10000,탁주,8.0,백구 강아지 달콤하면서 색다른 고소한 유기견 크라운 시나몬 심볼 후원
3,10004,가야 프리미엄 막걸리,10000,탁주,6.0,김해 전혀 가야 달콤한 조량
4,10005,가와지탁주 7.5,10000,탁주,7.5,산뜻한 탄산 대화 아름다운 물씬 미세한 텍스쳐 산미 사과 자부심 형상 지라 바나나 ...
...,...,...,...,...,...,...
952,50086,호담 300ml,50000,리큐르/기타주류,20.0,산삼 대농 청정 산쌀 리큐르 통째 면적 호담 바이오 조합원 뿌리 산양
953,50087,홍삼명주,50000,리큐르/기타주류,14.0,홍삼 토양 기한 유통 명주 산화 주입 브랜드 내기 토질 충북 괴산 건강한 기다림 표...
954,50088,SSOA-포도,50000,리큐르/기타주류,5.0,아주 뛰어난 동시 스파클링 탄산 포도주 시원함을 포천 우수한 포도 특유한
955,50089,공주 알밤 왕밤주,50000,리큐르/기타주류,6.0,공주 진심 주가 달달 고소한 소비 완전 알밤 저온 순하고 상품화 부드러운 왕밤 고소...


- 연령 입력값 변수 : submit_age / (str)
- 성별 입력값 변수 : submit_gender / (str)
----------------------------------
- 주종 입력값 변수 : submit_type / (str)
- 도수 입력값 변수 : submit_proof / (str)
- 선택 키워드 입력값 변수 : submit_keyword / (str)
----------------------------------
- 유사도 검사 결과 프레임(20개행) : simil_result / (dataframe)

In [90]:
# 예시
submit_type = '증류주'
submit_proof = '25 up'

In [106]:
def filtering(submit_type, submit_proof):
    
    first_filter = tb_join[tb_join['TYPE_NAME'] == submit_type]
    val = 'up' in submit_proof
    proof = float(re.sub(r'[^0-9]', '', submit_proof))

    if val : 
        second_filter = first_filter[first_filter['PRODUCT_PROOF'] >= proof]
    else : 
        second_filter = first_filter[first_filter['PRODUCT_PROOF'] <= proof]
    
    recommend_ID = second_filter.head(1)['PRODUCT_ID'].values[0]
    
    return recommend_ID

In [107]:
filtering(submit_type, submit_proof)

'20001'

In [108]:
first_filter = tb_join[tb_join['TYPE_NAME'] == submit_type]

In [109]:
first_filter

,PRODUCT_ID,PRODUCT_NAME,TYPE_ID,TYPE_NAME,PRODUCT_PROOF,INFO_KEYWORD
346,20001,겨울소주 25,20000,증류주,25.0,부드럽고 아리랑 부드럽다 조의 가볍게 빚기 따끈한 감압 요리 비롯 다음 고소하고 적...
347,20002,고소리술 29,20000,증류주,29.0,부드럽고 처음 최우수상 누룽지 고리 화려한 제주도 오메 장려상 명성 단맛 소리 번의...
348,20003,고운달 오크,20000,증류주,52.0,쉬운 부드럽고 오크 은은하게 아니지만 더하는 화려한 결코 포인트 속성 형상 고운 감...
349,20004,금설,20000,증류주,35.0,금박 부드럽고 선사 싸라기눈 멋진 금설 식용 살짝 모두 만족감 깔끔하며 내면 금빛 ...
350,20005,담 다이아몬드,20000,증류주,40.0,과실 반짝 깨끗하면서 은은한 맑고 달콤하고 우아한 다이아몬드 투명하게 표현 거북하지...
...,...,...,...,...,...,...
542,20201,주향아라,20000,증류주,55.0,부드럽고 처럼 그냥 병입 화려한 위스키 숙성해 얼음 도자기 점점 가능한 주향 아리 ...
543,20202,주향이오,20000,증류주,25.0,이오 고소하고 부드럽고 곡향 편하게 숙성해 브랜디 처럼 항아리 유순하게 운영 은은한...
544,20203,키(KHEE) 22,20000,증류주,22.0,배우 마케팅 뉴욕타임스 예술가 유명 카운티 수로 미술관 화제 론칭 신세계 이정재 재...
545,20204,키(KHEE) 38,20000,증류주,38.0,배우 미네랄 성한 마케팅 뉴욕타임스 예술가 유명 카운티 미술관 화제 남아 론칭 신세...


In [110]:
val = 'up' in submit_proof
proof = float(re.sub(r'[^0-9]', '', submit_proof))

if val : 
    second_filter = first_filter[first_filter['PRODUCT_PROOF'] >= proof]
else : 
    second_filter = first_filter[first_filter['PRODUCT_PROOF'] <= proof]

In [111]:
second_filter

,PRODUCT_ID,PRODUCT_NAME,TYPE_ID,TYPE_NAME,PRODUCT_PROOF,INFO_KEYWORD
346,20001,겨울소주 25,20000,증류주,25.0,부드럽고 아리랑 부드럽다 조의 가볍게 빚기 따끈한 감압 요리 비롯 다음 고소하고 적...
347,20002,고소리술 29,20000,증류주,29.0,부드럽고 처음 최우수상 누룽지 고리 화려한 제주도 오메 장려상 명성 단맛 소리 번의...
348,20003,고운달 오크,20000,증류주,52.0,쉬운 부드럽고 오크 은은하게 아니지만 더하는 화려한 결코 포인트 속성 형상 고운 감...
349,20004,금설,20000,증류주,35.0,금박 부드럽고 선사 싸라기눈 멋진 금설 식용 살짝 모두 만족감 깔끔하며 내면 금빛 ...
350,20005,담 다이아몬드,20000,증류주,40.0,과실 반짝 깨끗하면서 은은한 맑고 달콤하고 우아한 다이아몬드 투명하게 표현 거북하지...
...,...,...,...,...,...,...
541,20200,주향담을,20000,증류주,41.0,처럼 부부 부드러우며 약간 화려한 부드러움 깔끔하면서 속성 위스키 순수 숙성해 부드...
542,20201,주향아라,20000,증류주,55.0,부드럽고 처럼 그냥 병입 화려한 위스키 숙성해 얼음 도자기 점점 가능한 주향 아리 ...
543,20202,주향이오,20000,증류주,25.0,이오 고소하고 부드럽고 곡향 편하게 숙성해 브랜디 처럼 항아리 유순하게 운영 은은한...
545,20204,키(KHEE) 38,20000,증류주,38.0,배우 미네랄 성한 마케팅 뉴욕타임스 예술가 유명 카운티 미술관 화제 남아 론칭 신세...


In [112]:
second_filter.head(1)

,PRODUCT_ID,PRODUCT_NAME,TYPE_ID,TYPE_NAME,PRODUCT_PROOF,INFO_KEYWORD
346,20001,겨울소주 25,20000,증류주,25.0,부드럽고 아리랑 부드럽다 조의 가볍게 빚기 따끈한 감압 요리 비롯 다음 고소하고 적...
